In [ ]:
#import zipfile
#with zipfile.ZipFile('spotify_million_playlist_dataset.zip', 'r') as zip_ref:
    #zip_ref.extractall('Spotify MPD')

In [ ]:
pip install pandas

In [ ]:
pip install spotipy

In [ ]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [ ]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [ ]:
############################# V3 - BATCH V2 ###################
# TODO: TIME THIS TO SEE IF IT WORKS
# TODO: CALCULATE THE OTHER TWO METRICS
# TODO: CLEAN UP DATES (divide by something, make the values way smaller and potentially all positive)
# TODO: WHY IS tempo_var SO HIGH?

In [ ]:
# get all track URIs for all playlists
def get_all_track_URIs(playlists):
    uri_list = []
    for playlist in playlists:
        playlist = pd.DataFrame(playlist)
        #print(playlist)
        uri_list.append(playlist['track_uri'])
        #for tracks in playlist['tracks']:
            #uri_list += tracks['track_uri']
    #print(np.shape(uri_list))
    return uri_list
    #data['playlists'][0]['tracks'][i]['track_uri']
    #print(uri_list)
#all_uris = get_all_track_URIs(data['playlists'])
#print(all_uris[0:2])

In [ ]:
pd_data = pd.DataFrame(data['playlists'])
#print(pd.DataFrame(pd_data['tracks'][0])['track_uri'])
print(pd.DataFrame(pd_data['tracks']))
print(np.shape(pd_data['tracks']))
all_uri_attempt = pd.DataFrame(pd_data['tracks'])['track_uri']
print(np.shape(all_uri_attempt))

In [ ]:
print(list(pd_data['tracks'][0][0].values())[2])
# uri index in track object is 2


In [ ]:
# get all audio features
def get_all_audio_features_batch(playlist_uris):
    all_features = []
    for playlist in playlist_uris:
        remainder = len(playlist) % 100
        if remainder > 0: 
            all_features.append(sp.audio_features(playlist[:remainder]))
        while remainder < len(playlist):
            #print(all_features[-1])
            all_features[-1] += sp.audio_features(playlist[remainder:remainder+100])
            remainder += 100
    return all_features

In [ ]:
# get all aggregate audio features, build the dataset
def get_all_aggregate_features(playlists):
    all_playlist_uris = get_all_track_URIs(playlists)
    all_features = get_all_audio_features_batch(all_playlist_uris)
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    all_aggregate_features = []
    for playlist in all_features:
        playlist = pd.DataFrame(playlist)
        aggregate_features = {}
        for feature in feature_list:
            #all_by_feature = [track[feature] for track in all_features]
            all_by_feature = playlist[feature]
            aggregate_features[feature] = np.mean(all_by_feature)
            aggregate_features[feature+'_var'] = np.var(all_by_feature)
        all_aggregate_features.append(aggregate_features)
    #print(np.shape(aggregate_features))
    return all_aggregate_features

In [ ]:
pd_data = pd.DataFrame(data['playlists'])
t = timeit.Timer('get_all_aggregate_features(pd_data[\'tracks\'][0:100])', 'from __main__ import get_all_aggregate_features, pd_data')
print('V3: ' + str(t.timeit(1)))
t = timeit.Timer('build_dataset_v2(data[\'playlists\'][0:100])', 'from __main__ import build_dataset_v2, data')
print('V2: ' + str(t.timeit(1)))

In [ ]:
pd_data = pd.DataFrame(data['playlists'])
dataset = get_all_aggregate_features(pd_data['tracks'][0:4])

In [ ]:
############################### VERSION 1 #######################################

In [ ]:
# get all track URIs for a given playlist
def get_uris_for_playlist(tracks):
    uri_list = []
    for track in tracks:
        uri_list.append(track['track_uri'])
    return uri_list
#playlist_uris = get_uris_for_playlist(data['playlists'][0])

# get all audio features for a given playlist
def get_playlist_audio_features(tracks):
    uri_list = get_uris_for_playlist(tracks)
    all_features = []
    for uri in uri_list:
        track_features = sp.audio_features(uri) # consider batching this
        # this will have unnecessary features, will be removed in aggregation step
        all_features.append(track_features[0])
    #print(all_features[0])
    return all_features

# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features(tracks):
    all_features = get_playlist_audio_features(tracks)
    #print(all_features[0:5])
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = []
        for track in all_features:
            #print(track)
            all_by_feature.append(track[feature])
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    return aggregate_features

# build the aggregate dataset!
def build_dataset(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features(playlist['tracks']))
    return aggregate_dataset

In [ ]:
#######################################################################

In [ ]:
# get all audio features for a given playlist
def get_playlist_audio_features_v2(tracks):
    all_features = []
    for track in tracks:
        track_features = sp.audio_features(track['track_uri']) # consider batching this
        # this will have unnecessary features, will be removed in aggregation step
        all_features.append(track_features[0])
    #print(all_features[0])
    return all_features

In [ ]:
################################## V2 - BATCH V1 ######################################

In [ ]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(tracks):
    tracks = pd.DataFrame(tracks)
    playlist_uris = tracks['track_uri']

    remainder = len(tracks) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(tracks):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    remainder = len(tracks) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates = get_artists_and_dates(playlist_uris[:remainder])
    while remainder < len(tracks):
        next_artists, next_dates = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
    return all_features, artists, dates

In [ ]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features_v2(tracks):
    all_features, artists, dates = get_playlist_audio_features_batch(tracks)
    all_features = pd.DataFrame(all_features)
    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        #all_by_feature = [track[feature] for track in all_features]
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    # calculate artist index - right now doesn't account for featured artists
    # can account for featured artists using data from sp.tracks() method
    #aggregate_features['artist_conc'] = num_artists / len(tracks)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    return aggregate_features

In [ ]:
# build the aggregate dataset!
def build_dataset_v2(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features_v2(playlist['tracks']))
    return aggregate_dataset

In [ ]:
playlist1 = tracks_test
playlist1_df = pd.DataFrame(playlist1)

def test_playlist1(playlist1):
    uris = []
    for track in playlist1:
        uris.append(track['track_uri'])
    return uris


def test_playlist1_df(playlist1_df):
    return playlist1_df['track_uri']

In [ ]:
t = timeit.Timer('test = test_playlist1(playlist1)', 'from __main__ import test_playlist1, playlist1')
print('no DF: ' + str(t.timeit(1)))
t = timeit.Timer('test = test_playlist1_df(playlist1_df)', 'from __main__ import test_playlist1_df, playlist1_df')
print('DF: ' + str(t.timeit(1)))

In [ ]:
tracks_prep = [playlist['tracks'] for playlist in data['playlists'][:20]]
tracks_test = []
for playlist in tracks_prep:
    for track in playlist:
        tracks_test.append(track)
print(np.shape(tracks_test))

In [ ]:
t = timeit.Timer('get_playlist_audio_features(tracks_test)', 'from __main__ import get_playlist_audio_features, tracks_test')
print('V1: ' + str(t.timeit(1)))
t = timeit.Timer('get_playlist_audio_features_v2(tracks_test)', 'from __main__ import get_playlist_audio_features_v2, tracks_test')
print('V2: ' + str(t.timeit(1)))

In [ ]:
dataset = build_dataset_v2(data['playlists'][434:437])
#print(dataset)

In [ ]:
# sp.audio_features(tracks=[]) # max 100 ids
# sp.tracks(tracks=[], market=None) # max 50 ids

In [ ]:
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    #old_dates = []
    tracks = sp.tracks(uris)['tracks']
    #print(tracks)
    #albums = tracks['album']
    #print(tracks['artists'][0])
    # TODO: FIGURE OUT HOW TO DO THIS MORE EFFICIENTLY
    #print(len(tracks))
    for track in tracks:
        if track is None:
            print(track)
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            dates.append(dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))) # some dates are negative bc pre-1970, does that matter?
        #old_dates.append(track['album']['release_date'])
    #print('############################################')
    #print(artists)
    #print(len(artists))
    #print('############################################')
    #print(dates)
    #print(old_dates)
    return artists, dates

In [ ]:
print(data['playlists'][497:498])

In [ ]:
#t = timeit.Timer('build_dataset(data[\'playlists\'][0:3])', 'from __main__ import build_dataset, data')
#print('V1: ' + str(t.timeit(1)))
#print(data['playlists'][0:4])
t = timeit.Timer('build_dataset_v2(data[\'playlists\'][:100])', 'from __main__ import build_dataset_v2, data')
print('V2: ' + str(t.timeit(1)))
# right now it's 4.6 days for the whole thing
# 400 seconds for the whole dataset
# 35 seconds ish for the first hundred

In [ ]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    return artist_conc

In [ ]:
dataset = build_dataset_v2(data['playlists'][1:2])
#print(dataset)
#data['playlists'][434:435][0]['num_artists']

In [ ]:
first_tracks = sp.tracks(pd.DataFrame(tracks_test)['track_uri'][:50])

In [ ]:
tracks = pd.DataFrame(first_tracks['tracks'])
albums = pd.DataFrame(tracks['album'])
#print(albums['release_date'])
print(pd.DataFrame(albums['album']))
print(np.shape(albums))

#print(pd.DataFrame(pd.DataFrame(first_tracks['tracks'])['album'])[0:2]['release_date'])
#print(type(pd.DataFrame(first_tracks['tracks'])['album'][0]['release_date']))

In [ ]:
print(pd.DataFrame(pd.DataFrame(first_tracks['tracks'])['artists'][0])['name'])

In [ ]:
name_str = 'mpd_slice_'
num1_str = '0_'
num2_str = '999.json'
file_str = name_str + num1_str + num2_str
print(file_str)
num1_str = '000_'
dataset = []
for i in range(1,1000):
    file_str = name_str + str(i) + num1_str + str(i) + num2_str
    print(file_str)
    with open(file_str) as json_file:
        data = json.load(json_file)
    dataset += build_dataset_v2(data['playlists'])